<a href="https://colab.research.google.com/github/ian-byrne/MADSmilestone2/blob/main/ian_testingground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook used to test project ideas
Author: Ian Byrne

Email: ianbyrne@umich.edu

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Reading in data stream

In [2]:
# Clone the entire repo.
!git clone -l -s https://github.com/ian-byrne/MADSmilestone2.git

# Change directory into cloned repo
%cd MADSmilestone2

# List repo contents
# !ls

fatal: destination path 'MADSmilestone2' already exists and is not an empty directory.
/content/MADSmilestone2


In [3]:
!ls

Cleaning_labeling.ipynb  Images.ipynb  Labeling  README.md
ian_testingground.ipynb  Images.py     Loading


Load imports as necessary

In [4]:
!pip install boto3

In [5]:
import ast
import boto3
import pandas as pd
import numpy as np
from PIL import Image
import Loading.load_data as loading
import torch
from torch.utils.data import Dataset, DataLoader

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


 Connect to S3 using the python client boto3
 - Access keys are loaded from clockss3 which is contained within my loacl google drive. 

In [9]:
# also import keys for aws connection
from gdrive.MyDrive.Colab_Notebooks.clocks_aws_config import clockss3
pubkey = clockss3['accessCode']
seckey = clockss3['secretCode']

In [10]:
client = boto3.client('s3', aws_access_key_id=pubkey, aws_secret_access_key=seckey)
response = client.list_buckets()
# response['Buckets']

In [15]:
s3 = boto3.resource('s3', aws_access_key_id=pubkey, aws_secret_access_key=seckey)
list(s3.buckets.all())[3]

s3.Bucket(name='clockdrawingimages1')

In [16]:
bucket = s3.Bucket('clockdrawingimages1')

In [17]:
list(bucket.objects.all())[0:5]

[s3.ObjectSummary(bucket_name='clockdrawingimages1', key='10000003.tif'),
 s3.ObjectSummary(bucket_name='clockdrawingimages1', key='10000004.tif'),
 s3.ObjectSummary(bucket_name='clockdrawingimages1', key='10000005.tif'),
 s3.ObjectSummary(bucket_name='clockdrawingimages1', key='10000006.tif'),
 s3.ObjectSummary(bucket_name='clockdrawingimages1', key='10000008.tif')]

~~Import dataframe containing spid, description of clock, diagnosis, round~~

Import the text file containing the dictionary of rounds and their spids and lebels. 
- Transform into a nested list to make loops easier. 

In [18]:
# rounds_df = loading.load_data()
# rounds_df.to_csv("round_data.csv", index = False)
# rounds_df.head()

In [19]:
file = open("Labeling/roundIdLabelDict.txt", "r")

contents = file.read()
round_dict = ast.literal_eval(contents)

In [20]:
test_dict = {1: [('10000003', 2)], 
             2: [('10000940', 2)], 
             3: [('10011363', 1)], 
             4: [('10010317', 1)]}

In [45]:
for tup in test_dict[1]:
    print(type(tup[0]), tup[1])

<class 'str'> 2


### Custom data set, idea:
- take in the dataframe from the rounds_df
- utiilize the spid number and the round number to call S3 dynamically and retrieve the correct URL for the .tif image
- tbd: any transforms to be done on the images to prepare for learning
    - Can likely ultilize this to loop through the large images and convert to smaller images and then save them to a new directory/ or locally to make the data more manageable 
- Should return the image in tensor format and the label (diagnosis)

In [43]:
class ClockImages(Dataset):
    """Original large format clock drawing dataset"""

    def __init__(self, round, round_labels):
        """
        Args:
            round (int): Round to grab images from. 
            values (list of tuples): Corresponding values for the round.
        """
        self.round = round
        self.vals = round_labels
        self.client = boto3.client('s3', aws_access_key_id=pubkey, aws_secret_access_key=seckey)


    def __len__(self):
        return len(self.vals)

    
    def __getitem__(self, idx):
        spid = self.vals[idx][0]
        label = self.vals[idx][1]
        bucket = f"ClockDrawing{self.round}"
        obj_name = f"{spid}.tif"
        
        im_obj = client.download_file(bucket, obj_name, obj_name)

        im = Image.open(im_obj)
        small_im = im.resize(207, 160)
        img = np.array(small_im)

        sample = {'image': img, 'label': label}

        return sample



In [54]:
data = ClockImages(1, test_dict[1])
print(data.round)
print(data.vals)
print(data.vals[0][0])
print(data.vals[0][1])

1
[('10000003', 2)]
10000003
2


In [46]:
clock_dataloader = torch.utils.data.DataLoader(dataset=data)
for sample in clock_dataloader:
    print(sample) # idk what this will output but if it works it should output something 

ClientError: ignored

In [23]:
# url = 'https://clockimages.s3.us-west-1.amazonaws.com/NHATS_R' + str(
#                     id) + '_ClockDrawings/' + value + '.tif'